**Find Trends in the data for modeling forward pass**

Similar to `experiments/step_time_analysis.ipynb` we are using current batch data we gathered in profiling to estimate forward pass times with a model

In [ ]:
from utils import parse_csv, format_data, concatenate_dataframes # found in utils.py

path1 = './profiling/train'

# # parse and concat all dataframes
df = concatenate_dataframes(path1)
df = format_data(df)
# df.to_csv('formatted_execution_stats500.csv', index=False)

In [9]:
# try to find correlation between execute_time and num_decodes, num_prefills, sum_decode_tokens, sum_prefill_tokens
correlation = df[['execute_time', 'num_prefills', 'sum_decode_tokens', 'sum_prefill_tokens', 'max_prefill_tokens']].corr()
print(correlation)

                    execute_time  num_prefills  sum_decode_tokens  \
execute_time            1.000000      0.804904           0.860769   
num_prefills            0.804904      1.000000           0.596136   
sum_decode_tokens       0.860769      0.596136           1.000000   
sum_prefill_tokens      0.753547      0.666530           0.434525   
max_prefill_tokens      0.736851      0.603709           0.439435   

                    sum_prefill_tokens  max_prefill_tokens  
execute_time                  0.753547            0.736851  
num_prefills                  0.666530            0.603709  
sum_decode_tokens             0.434525            0.439435  
sum_prefill_tokens            1.000000            0.964966  
max_prefill_tokens            0.964966            1.000000  


In [ ]:
# train a linear regression model to predict execute_time based on num_prefills, sum_decode_tokens, sum_prefill_tokens, and arrival_rate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X = df[['sum_decode_tokens', 'sum_prefill_tokens', 'max_prefill_tokens','num_prefills']]
y = df['execute_time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)
print("Score:", model.score(X_test, y_test))
# make predictions on the test set

Coefficients: [7.00524792e-05 1.95214710e-08 1.09916161e-08 2.84607896e-03]
Intercept: 0.004977576410358687
Score: 0.9462946728871697
